# The Solutions
### (to the xAI workshop, by Jack Fraser-Govil)

This document serves as the solutions sheet to the [main workshop notebook](../xai_workshop.ipynb). It is an almost exact duplicate of that file, with the code sections filled in such that the tasks are completed. 

##### However

Whilst it is certainly possible for you to copy and paste the entire contents of this file into your worksheet, finish all the tasks in 10 minutes, and then go for an early lunch, the entire purpose of this workshop is for you to come away with the satisfaction of having written your own learning networks from scratch. We are, by definition, reinventing the wheel here: everything we make will be orders of magnitude worse than pytorch or tensorflow; the principle is that **you learn best by doing it yourself**.

If you're going to use the 'cheat sheet', then try, first of all, to simply look at it to understand what it is doing, and then implement your own version. Don't directly copy whole chunks of code you don't understand, as that sort of defeats the point of today's efforts.



## Exercise 1: The Pretrained Perceptron

Just to get the juices flowing, let's get started with an easy project.

**In the ```Data``` directory is a file ```cuteness.tex```, which contains (non-dimensional) measures of size and furriness of some animals, as well as some human-curated labels (1 for cute, 0 for note cute). Please write a method which performs Perceptron classification, and use it to identify which of these creatures the model currently mislabels.**  

I've provided some basic functions and a class framework for you to work from.

In [114]:
#### This cell is unmodified form the version in the workshop notebook, it's just here so that this notebook can run as a standalone

# !git clone https://github.com/wtsi-hpag/xAIWorkshop.git
# %cd xAIWorkshop/Code

# This is the extent of fancy libraries we will be using
import numpy as np
import matplotlib.pyplot as pt

##load and parse the data manually. No pandas to be seen here! This function will be used repeatedly to load in our test data. 
def LoadCategoricalData(file):
	names = []
	labels = []
	data = []
	with open(file) as file:
		for line in file:
			entries = line.rstrip().split(' ')
			names.append(entries[0])
			labels.append(int(entries[1]))
			data.append(np.array([ float(entries[i]) for i in range(2,len(entries))]))
	return names, labels, data

names, labels, data = LoadCategoricalData("../Data/cuteness.dat")

In [ ]:
#### This cell is unmodified form the version in the workshop notebook, it's just here so that this notebook can run as a standalone

def PlotData(positions, labels,names=None):
	labels = [int(b) for b in labels] ## just to make sure the data is ints we can index in, not bools
	cols = ["red","blue"]
	x,y = zip(*positions)
	pt.scatter(x,y,color=np.array(cols)[labels])
	if names is not None:
		for i,pos in enumerate(positions):
			pt.annotate(names[i],pos)
	pt.show()

pt.title("True Assignments")
pt.xlabel("Size")
pt.ylabel("Fur")
PlotData(data,labels,names)

In [116]:
class Perceptron:

	def __init__(self):
		self.Weights = np.array([1,1,-1]) ## we'll use these pre-provided weights for now


	#####################################
	def Predict(self,input):
		
		## this is just a fancy bit so that it can (inefficiently) loop over an input list and return a list of predictions.
		if type(input) is list:
			out = []
			for r in input:
				# print(r)
				out.append(self.Predict(r))
			return out
		
		## this is the meat of the predictor
		return 1.0 * (np.dot(self.Weights,np.insert(input,0,1)) > 0)
	
		## this is a less one-line version of that same code:
		# augmentedVector = np.inserp(input,0,1) ##augment to add in the bias term -- weights have len = 3, but input is (x,y).
		# dot = np.dot(self.Weights,augmentedVector)
		# label = dot > 0 ## this is a boolean value
		# return 1.0 * label ##convert to 0.0 (false) or 1.0 (true) because plotting libraries don't like booleans! 

	#####################################


In [ ]:
#just a quick analysis function which we can re-use.
def AnalysePerceptron(Perceptron,data,labels,names=None,plotting = False):

	

	# this generates an array of predicted labels
	predict_labels = np.round(Perceptron.Predict(data)) ## rounding is for later!
	#this works out which ones are incorrect then prints them
	misslabels = predict_labels != np.array(labels)
	if names is not None:
		missed_names = np.array(names)[misslabels]
		print("Mislabelled entities are:", missed_names)
        
        
	print(f"Success Rate= {round(100*(1.0 - np.sum(misslabels) * 1.0/len(names)))}%")

	if plotting:
		## this is an anlytical means of converting the weights into y = mx + c
		x = np.linspace(0,10,10)
		w = P.Weights
		y = -w[1]*x/w[2] - w[0]/w[2]
		pt.plot(x,y)
		#this then plots the data with blue = correct predictions, red = false predictions
		PlotData(data,(predict_labels==np.array(labels)),names)
	

P = Perceptron()
AnalysePerceptron(P,data,labels,names,True)

## Exercise 2: Training The Perceptron

You should (hopefully) now have a working Perceptron; the only issues is a) it's absolutely terrible and b) I had to give you the weights. 

The next challenge is to write up a training algorithm for the Perceptron. 

**Add a method (Train) to the Perceptron, which iterates over the dataset a number of times (you choose - when is 'enough'?) and updates the weights**

Remember, given a datum $\mathbf{x}$, a prediction $P$ and the correct label $L$ the update formula is:

\begin{equation}
	\mathbf{w} \to \mathbf{w} + r \times \left(L - P(\mathbf{x}) \right)\mathbf{x}
\end{equation}
Where $r$ is your learning rate.

In [ ]:
## I'm fully redefining the Perceptron class; just so that these notebook is iteratively solved. You don't have to do that!
class Perceptron:

	def __init__(self):
		self.Weights = np.array([1.0,1,-1]) ## we'll use these pre-provided weights for now

	def Predict(self,input):
		
		if type(input) is list:
			out = []
			for r in input:
				# print(r)
				out.append(self.Predict(r))
			return out
	
		## this only performs the augmentation if the input dimensions are of the incorrect size; this permits pre-augmenting your data. 
		if len(input) == len(self.Weights):
			return 1.0 * (np.dot(self.Weights,input) > 0)
		else:
			return 1.0 * (np.dot(self.Weights,np.insert(input,0,1)) > 0)

	def Train(self, data,labels,fullEpochs = 100):

		rate = 0.01
		augmentedData = []
		for d in data:
			augmentedData.append(np.insert(d,0,1)) ## preaugment my data so that I don't waste CPU cycles doing it over and over again!

		ids = np.arange(len(data))
		for l in range(fullEpochs):
			noMistakes = True
			np.random.shuffle(ids) ## go through the data in a different order each time. Helps convergence
			for r in range(len(data)):

				P = self.Predict(augmentedData[ids[r]])

				updateSize = rate*(labels[ids[r]]-P) 
				update = updateSize * augmentedData[ids[r]]
				self.Weights +=  update
				if (P!= labels[ids[r]]):
					noMistakes = False
			if noMistakes:
				print("Converged early, after",l,"epochs")
				return

P = Perceptron()
P.Train(data,labels,500)
AnalysePerceptron(P,data,labels,names,True)



## Exercise 3: The Wrench in the Works

If everything has gone to plan, you now have an algorithm that can correctly identify whether a given animal is cute or note, based on measures of their size and amount of fur. 

Now try running your Perceptron training & testing routines on a new dataset: ```cuteness_augmented```.

**What goes wrong?**

In [ ]:
names_aug, labels_aug, data_aug = LoadCategoricalData("../Data/cuteness_augmented.dat")

P = Perceptron()

sum = 0
for train in [0,100,200,300,1000,195,107,206]:
	P.Train(data_aug,labels_aug,train)
	pt.title(f"After {sum} Epochs")
	AnalysePerceptron(P,data_aug,labels_aug,names_aug,True)
	sum +=train


### Optional Task: The Nonlinear Perceptron

It is perfectly possible to generate a Perceptron which can deal with these difficult and non-linear cases: the trick is that you have to put the nonlinearity in yourself. 

If you have time, try the following:

**Write a nonlinear perceptron which transforms the input vector (x,y) into a nonlinear vector (such as (x,y,x^2,y^3 sin(x))), and then uses this as the vector in a standard Perceptron algorithm. Does it show any improvements? How much work does it take to make the algorithm perfectly separate our cute animals from the uggos?**

In [ ]:
from tqdm import tqdm #ok I lied about no more libraries, but I wanted a progress bar
class NonlinearPerceptron:

	def __init__(self,maxPower=1):
		self.Power = maxPower
		self.nDim = int(((maxPower+2) * (maxPower + 1))/2)
		self.Weights = np.zeros((self.nDim,)) ## we'll use these pre-provided weights for now
		self.Weights[0] = 0.5
		self.Weights[1] = 1
		self.Weights[2] = -1

		# print(self.Weights)
	def Predict(self,input,vectorfied=False):
		
		if type(input) is list:
			out = []
			for r in input:
				# print(r)
				out.append(self.Predict(r))
			return out
		if vectorfied:
			return 1.0 * (np.dot(self.Weights,input) > 0)
		else:
			vec = self.Vectorfy(input)
			return 1.0 * (np.dot(self.Weights,vec) > 0)

	## this is the non-linear magic bit. It generates the set of all 2D polybinomials up to the specified order.
	def Vectorfy(self,input):
		
		output = np.zeros((self.nDim,))
		output[0] = 1
		s = 1
		
		for power in range(1,self.Power+1):
			for q in range(power+1):
				output[s] = input[0]**q * input[1]**(power-q)
				s+= 1
		return np.array(output)


	def Train(self, data,labels,fullEpochs = 100):
		vectorfied = [self.Vectorfy(d) for d in data]


		rate = 0.5
		##this is a bit of a hack (and a hint at better optimisation routines!)
		## Because my vector has higher powers of x, the optimiser wobbles incoherently if the powers are x**10 as it's simultaenoulsy trying to balance +/- 1 and +/- a billion
		## this slows down the learning at the higher power parameters, letting it be much less sensitive to them
		rates = np.zeros((self.nDim))
		s = 0
		for power in range(self.Power):
			for r in range(power+1):
				rates[s] = rate / 5**power
				s+= 1

		ids = np.arange(len(data))
		for l in tqdm(range(fullEpochs)):
			np.random.shuffle(ids)
			noMistakes = True
			for r in range(len(data)):
				P = self.Predict(vectorfied[ids[r]],True)

				update = rates*(labels[ids[r]]-P) * vectorfied[ids[r]]
				self.Weights +=  update
				if (P!= labels[ids[r]]):
					noMistakes = False
			if noMistakes:
				print("Converged early, after",l,"epochs")
				return


## this function lets me plot surface maps of where the predictor has drawn its decision boundaries.
def ScanClassifierSpace(Predictor,xmin,xmax,ymin,ymax,resolution,dotSize =4,cmapping=True):
	x = np.linspace(xmin,xmax,resolution)
	y = np.linspace(ymin,ymax,resolution)

	N = len(x)*len(y)
	xs = np.zeros((N,))
	ys = np.zeros((N,))
	zs = np.zeros((N,))
	s= 0
	z = np.zeros((len(y),len(x)))
	for i,xp in enumerate(x):
		for j,yp in enumerate(y):
			l = Predictor.Predict(np.array([xp,yp]))
			xs[s] = xp
			ys[s] = yp
			zs[s] = float(l)
			s+=1

	if cmapping:
		pt.scatter(xs,ys,dotSize,zs,cmap='RdBu')
	else:
		pt.scatter(xs,ys,dotSize,zs)


P = NonlinearPerceptron(7)
P.Train(data_aug,labels_aug,120000)

AnalysePerceptron(P,data_aug,labels_aug,names_aug,False)


ScanClassifierSpace(P,0,10,0,6,100)
pt.xlabel("Size")
pt.ylabel("Fur")
pt.clim([-0.2,1.2])
PlotData(data_aug,np.array(labels_aug),names_aug)

pt.show()




### Optional Task: The XOR Problem

If you've managed to get a non-linear Perceptron working, you might be feeling pretty pleased with yourself! You've managed to (with a single node, in modern terminology) fit a highly nonlinear classifier.

**In the Data directory are a series of datasets called ```XOR_n``` (where ```n``` ranges from 10 to 1000). Try fitting these with your non-linear classifier. What goes wrong?**

In [ ]:
names, labels, data = LoadCategoricalData("../Data/XOR_160.dat")
P = NonlinearPerceptron(6)
P.Train(data,labels,50)

ScanClassifierSpace(P,0,2,0,2,100)
pt.plot([0,2],[1,1],'k')
pt.plot([1,1],[0,2],'k')
pt.clim([-0.2,1.2])
pt.show()
PlotData(data,np.array(labels)) # plot on different axis because otherwise it gets a bit dense

## Exercise 3: A Multilayered Perceptron

We have seen that, although incredibly easy to implement and train, a Perceptron has fundamental limitations. In their normal form, they can only split a (hyperdimensional) plane into two parts; and even when we augment our input space with hand-crafted non-linearity, there are still some problems that it cannot solve. 

Let us assume, therefore, that the problem was *too few decision points*. The perceptron fundamentally, makes a choice if something is above or below a certain line. But some decisions are more complex than that. How can we let the Perceptron make more sub-decisions, before coming to a final conclusion? **A network of nodes!**

**Fill in the Network, Layer and Node classes such that they obey the following restrictions**
* A Network *has* a number of layers. 
* Each layer *has* a number of nodes. 
* Each node *has* an (individual) weight vector. 
* The dimension (length) of each weight vector is equal to the number of nodes in the previous layer. 
	* The dimension of the weight vector in the first layer is equal to the dimension of your data
* A Network takes as input a datum, and outputs a prediction. 
* A Layer takes as input the output from the previous layer (the first layer takes the network input)
	* The layer then gives that vector to each node, which performs $y_i = \mathbf{w}_i \cdot \begin{bmatrix} 1 \\ \mathbf{x} \end{bmatrix}$ on it. 
	* The layer then assembles each $y_i$ into a new vector $\mathbf{y}$. This is the output of the layer.
* The Network takes the output of the final layer and performs the usual Perceptron classifier on it: if the result is greater than 0 it returns "yes", if less than 0 it returns "no".

This sounds complex, but for now, is essentially just a complex series of handing vectors between the Network, the Layers and the Nodes. The nodes do a dot product, the Layers assemble that into a vector, and hand it off to the next layer. 

In [122]:

# A linear node is a simple perceptron node (without the >0 check). It's a dot product between the weights and the 
class Node:
	def __init__(self):
		self.Weights = np.zeros((0,)) ## initialise an empty vector -- we cannot populate it yet because we don't know how big it is. We only know when the Network has been assembled. 

	## this function informs the node of its dimensions, and is called by the Network & Layer after construction.
	def Initialise(self, dimension):
		self.Weights = 2*(np.random.random((dimension,)) - 0.5) # randomly initialises to a value between -1 and 1.
	
	def Predict(self, inputVector): ##I assume that the layer will augment for me, to save some CPU cycles
		return np.dot(self.Weights,inputVector)


## A layer is a collection of nodes. The layer takes an input vector, distributes them to the nodes, and then aggregates the data.
class Layer:
	def __init__(self,nodeCount, nodeType=Node):
		#nodeCount is the number of nodes in the data, and nodeType is a class object. Note that I have defined nodeType as the *class*, not a member of that class; this lets the layer call the initialiser itself, and avoid the copying issues python can sometimes encounter.

		## should initialise the nodes so that they contain nodeCount copies of the nodeType class. Be careful that you initialise the node (so call nodeType(), don't just put nodeType into your vector).
		self.Nodes = []
		for i in range(nodeCount):
			self.Nodes.append(nodeType())
		self.OutputVector = np.zeros((nodeCount,)) #rather than returning a vector and passing it around, I store it as a member of the class. Then I just slot things in rather than creating a new vector each loop. Saves some CPU cycles.
		
	def Initialise(self,previousLayerDimension):
		for node in self.Nodes:
			node.Initialise(previousLayerDimension)

	def Predict(self, inputVector):
		augmentedVector = np.insert(inputVector,0,1)
		for i,node in enumerate(self.Nodes):
			self.OutputVector[i] = node.Predict(augmentedVector)


## The Network class is your ``
class Network:
	def __init__(self, inputDimension, outputDimension=1):
		#input dimension is the dimension of your data -- so [x,y] is 2. 
		#output dimension is the dimension of your prediction -- for now this is a binary yes/no -- so one dimension. 

		self.Layers = [] #this is where your layers will go
		self.InputDimension = inputDimension
		self.OutputDimension = outputDimension

	def AddLayer(self, layerObject): #append a layer object into your network. 
		self.Layers.append(layerObject)

	def Initialise(self):
		##need an initialisation function because your network doesn't know (for instance) what dimension each of the weight vectors should be until it can see the final layer structure -- remember the dimension of the n-th layer weight vectors is equal to the number of nodes in the (n-1)th layer. 


		## this funciton should initialise each layer, giving them the correct information
		previousLayerDimension = self.InputDimension
		for layer in self.Layers:
			layer.Initialise(previousLayerDimension+1) ## the +1 is for the `bias` term. If you're manually including the bias, you don't need this
			previousLayerDimension = len(layer.Nodes)
	
		##then check that my dimensions make sense!
		if previousLayerDimension != self.OutputDimension:
			print("ERROR! Your network output dimension is not equal to the final layer dimension")



	def Predict(self, inputVector):

		self.Layers[0].Predict(inputVector)
		for i in range(1,len(self.Layers)):
			self.Layers[i].Predict(self.Layers[i-1].OutputVector)
		finalValue = self.Layers[-1].OutputVector
		##techincally this is a hack because I know that the dimension is 1 -- this is just for the curent challenge!
		return int(finalValue > 0)
	
	def Train(self, data,labels):

		#Leave this for now. It's going to be a big one!
		return


	
			

In [ ]:
# np.random.seed(1)

network = Network(2,1)
network.AddLayer(Layer(5))
network.AddLayer(Layer(3))
network.AddLayer(Layer(1))
network.Initialise()

## plot these out of bounds just to get the legend right!
pt.scatter([-100],[-100],5,'r',label='P = 0')
pt.scatter([-100],[100],5,'b',label='P=1')


ScanClassifierSpace(network,-10,10,-10,10,200)
pt.legend()
pt.xlim([-10,10])
pt.ylim([-10,10])
pt.xlabel("Size")
pt.ylabel("Fur")
pt.clim([-0.2,1.2])
pt.show()

### Exercise 5: Solving Our Problems

The problem identified above can *only* be solved by altering the way our neurons/nodes work.

**Write you own custom node activation function, then retest your network. Did this fix the problematic behaviour?**

In [ ]:
class SigmoidNode (Node):

	def Predict(self, inputVector):
		a = np.dot(self.Weights,inputVector)
		return 1.0/(1.0 + np.exp(-a))
	
np.random.seed(1)
network = Network(2,1)
network.AddLayer(Layer(5,SigmoidNode))
network.AddLayer(Layer(3,SigmoidNode))
network.AddLayer(Layer(1))
network.Initialise()

## plot these out of bounds just to get the legend right!
pt.scatter([-100],[-100],5,'r',label='P = 0')
pt.scatter([-100],[100],5,'b',label='P=1')

print(network.Predict([0,0]))
## then do my scan
ScanClassifierSpace(network,-10,10,-10,10,200)
pt.legend()
pt.xlim([-10,10])
pt.ylim([-10,10])
pt.xlabel("Size")
pt.ylabel("Fur")
pt.clim([-0.2,1.2])
pt.show()

## Training A Network

This is the big one. This is where our maths is going to *hurt*. We now want to try and make our network learn the parameters which make it work. 

Unlike the Perceptron, this 'learning' is best approached through the language of calculus. Unfortunately that means that we need to move away from our hard decision boundaries (which are non-continuous and therefore non-differentiable), and move to a smooth `cost function'. 

This cost function is small when a set of predictions is close to being correct, and large when it is far away. 

If we treat our output node as producing $P$, a *probability* of being cute (0 being 100% ugly, 1 being 100% cute), then a reasonable cost function could be something along the following lines:

\begin{equation}
	\mathcal{L} = \sum_{\text{data } i} \left(P_i - L_i\right)^2
\end{equation}
This least squares cost function meets our criteria: it is zero when the predictor guesses everything correctly ($P_i = L_i$), and it is larger than zero when it is wrong, or uncertain. 

We now simply need to find the set of weights / network parameters which optimize this function. This is a basic gradient descent task. Each weight can be updated according to the formula:
\begin{equation}
	\mathbf{w}_\ell^i \to \mathbf{w}_\ell^i - \frac{\alpha}{|\mathbf{d}_\ell^i|} \times \mathbf{d}_\ell^i
\end{equation}
Where
\begin{equation}
	\mathbf{d} = \frac{\partial \mathcal{L}}{\partial \mathbf{w}_\ell^i}
\end{equation}
This is basic Newtonian descent; you can equally use other methods such as Line Search or the ADAM optimiser, but whichever method you choose, ..we need the gradients. Rather than seeing our function $\mathcal{L}$ as a function of the input data conditioned on the weights, we treat it as a function of the weights, conditioned on the data:
\begin{equation}
	\mathcal{L} = \mathcal{L}(\{\vec{w} \}| \vec{d})
\end{equation}
Here $\{\vec{w}\}$ is the set of weights on each node. We need to find:
\begin{equation}
	\frac{\partial \mathcal{L}}{\partial \vec{w}_i} = 2\sum_i  \left(P_i - L_i\right) \frac{\partial P_i}{\partial \vec{w}}
\end{equation}

### Working Your Way Backwards

In the notes, I show that the following equations can be used to compute the gradients:

\begin{align}
\frac{\partial C_q}{\partial \vec{w}_\ell^i} &= \frac{\partial C_q}{\partial y_\ell^i} \tilde{\vec{x}}_{\ell-1}
\\
\frac{\partial C_q}{\partial y_\ell^i} & = \begin{cases} \frac{\partial C_q}{\partial P_q^i} \times d_N(y_N^i) & \text{ if $\ell = N-1$}
\\
d_\ell(y_\ell^i) \sum_{j}  \left[ \vec{w}_{\ell+1}^j \right]_{i+1} \frac{\partial C_q}{\partial y_{\ell+1}^j}
\end{cases}
\end{align}

This requires starting at the final layer ($\ell = N-1$) and working your way backwards through the network, using the previous layer's results to inform the next. For this reason it is known as *backpropagation*.

In [156]:
class CostFunction:
		
	def Compute(self,prediction,label):
		if type(prediction) is list:
			v = 0
			for p,i in enumerate(prediction):
				v += self.Compute(p,label[i])
			return v
		return (prediction - label)**2
	
	def Gradient(self,prediction,label):
		return 2 * (prediction - label)
	
class Node:
	def __init__(self):
		self.Weights = np.zeros((0,)) 
		

	def Initialise(self, dimension):
		self.Weights = np.random.normal(0,0.3,(dimension,))
		self.InternalGradient = np.zeros(self.Weights.shape)
		self.M = np.zeros(self.Weights.shape)
		self.V = np.zeros(self.Weights.shape)
		
		self.L = 1
		self.Y = 0
		self.Batch = 0
		self.dLdY = 0

	def Predict(self, inputVector): ##I assume that the layer will augment for me, to save some CPU cycles
		self.Y = np.dot(self.Weights,inputVector)
		return self.ActivationFunction(self.Y)
	
	def ActivationFunction(self, inputValue):
		return inputValue

	def ActivationGradient(self, inputValue):
		return 1
	
	def UpdateGradient(self, dLdY, inputVector):
		if not np.isinf(dLdY):
			self.dLdY = dLdY
			self.InternalGradient += self.dLdY * inputVector 
		self.Batch += 1
	def UpdateWeights(self,alpha):


		## this is the advanced update weight (the ADAM optimiser)
		b1 = 0.5
		b2 = 0.7
		c1 = 1.0/(1.0 - b1**self.L)
		c2 = 1.0/(1.0 - b2**self.L)
		self.L += 1
		
		self.InternalGradient/=self.Batch
		self.Batch = 0
		self.M = b1 * self.M + (1.0 - b1) * self.InternalGradient
		self.V = b2 * self.V + (1.0 - b2) * self.InternalGradient**2

		self.Weights -= alpha * (c1 *self.M)/np.sqrt(self.V*c2 + 1e-20)
		self.InternalGradient *= 0 ## clear for the next iteration!


		## this is the default
		# step = alpha * self.InternalGradient / np.linalg.norm(self.InternalGradient)
		# self.Weights -= step
		# self.InternalGradient *= 0 ## clear for the next iteration!

class SigmoidNode (Node):

	def ActivationFunction(self, y):
		return 1.0/(1.0 + np.exp(-y))
	def ActivationGradient(self, y):
		return np.exp(-y)/(1.0 + np.exp(-y))**2
class ReluNode (Node):

	def ActivationFunction(self, y):
		if y > 0:
			return y
		return 0.1*y
	def ActivationGradient(self, y):
		if y > 0:
			return 1
		return 0.1

## A layer is a collection of nodes. The layer takes an input vector, distributes them to the nodes, and then aggregates the data.
class Layer:
	def __init__(self,nodeCount, nodeType=Node):
		self.Nodes = []
		for i in range(nodeCount):
			self.Nodes.append(nodeType())
		self.OutputVector = np.zeros((nodeCount+1,))
		self.OutputVector[0] = 1
		
	def Initialise(self,previousLayerDimension):
		for node in self.Nodes:
			node.Initialise(previousLayerDimension)

	def Predict(self, inputVector):
		for i,node in enumerate(self.Nodes):
			self.OutputVector[i+1] = node.Predict(inputVector) ##offset for augmentation purposes!

	def MiddleLayerGradient(self,layerAbove,inputVector):
		for i,node in enumerate(self.Nodes):

			dLdy = 0
			for innernode in layerAbove.Nodes:
				dLdy += innernode.Weights[i+1] * innernode.dLdY
			dLdy *= node.ActivationGradient(node.Y)
			node.UpdateGradient(dLdy,inputVector)

	def FinalLayerGradient(self,dLdP,inputVector):
		for node in self.Nodes:
			node.UpdateGradient(dLdP * node.ActivationGradient(node.Y),inputVector)

	def UpdateWeights(self,alpha):
		for node in self.Nodes:
			node.UpdateWeights(alpha)
		
class Network:
	def __init__(self, inputDimension, outputDimension=1):
		#input dimension is the dimension of your data -- so [x,y] is 2. 
		#output dimension is the dimension of your prediction -- for now this is a binary yes/no -- so one dimension. 

		self.Layers = [] #this is where your layers will go
		self.InputDimension = inputDimension
		self.OutputDimension = outputDimension

	def AddLayer(self, layerObject): #append a layer object into your network. 
		self.Layers.append(layerObject)

	def Initialise(self):
		##need an initialisation function because your network doesn't know (for instance) what dimension each of the weight vectors should be until it can see the final layer structure -- remember the dimension of the n-th layer weight vectors is equal to the number of nodes in the (n-1)th layer. 


		## this funciton should initialise each layer, giving them the correct information
		previousLayerDimension = self.InputDimension
		for layer in self.Layers:
			layer.Initialise(previousLayerDimension+1) ## the +1 is for the `bias` term. If you're manually including the bias, you don't need this
			previousLayerDimension = len(layer.Nodes)
	
		##then check that my dimensions make sense!
		if previousLayerDimension != self.OutputDimension:
			print("ERROR! Your network output dimension is not equal to the final layer dimension")

	def Predict(self, inputVector):
		if len(inputVector) == self.InputDimension:
			inputVector = np.insert(inputVector,0,1)
		self.Layers[0].Predict(inputVector)
		for i in range(1,len(self.Layers)):
			self.Layers[i].Predict(self.Layers[i-1].OutputVector)
		return self.Layers[-1].OutputVector[1:] ##have to omit the augment from the final layer
	
	def Train(self, data,labels,costFunc = CostFunction(),Nsteps=1000):
		self.Initialise()

		N = len(data)
		augmentedData = []
		for i in range(N):
			augmentedData.append(np.insert(data[i],0,1)) #augment the data up front

		alpha = 0.1
		ids = np.arange(len(data))
		for steps in range(Nsteps):
			C = 0			
		
			for id in ids:
				p = self.Predict(augmentedData[id])
				C += costFunc.Compute(p,labels[id])

				dCdP = costFunc.Gradient(p,labels[id])
				self.Layers[-1].FinalLayerGradient(dCdP,self.Layers[-2].OutputVector)
				for j in np.arange(len(self.Layers)-2,-1,-1):
					if j > 0:
						self.Layers[j].MiddleLayerGradient(self.Layers[j+1],self.Layers[j-1].OutputVector)
					else:
						self.Layers[j].MiddleLayerGradient(self.Layers[j+1],augmentedData[id])

				
			for lNum,layer in enumerate(self.Layers):
				layer.UpdateWeights(alpha)
			if steps % 100 == 0:
				print(steps,"C=",C)
				alpha*=0.9				
		return
	
	## this is one of the advanced methods 
	def BatchTrain(self, data,labels,costFunc = CostFunction(),Nsteps=1000):
		self.Initialise()

		alpha = 0.1
		prevC = None
		meanC = 0
		N = len(data)
		augmentedData = []
		for i in range(N):
			augmentedData.append(np.insert(data[i],0,1)) #augment the data up front

		if len(data) > 100:
			batchSize = len(data)/100
		else:
			batchSize = len(data)
		ids = np.arange(N)
		q =0
		for steps in range(Nsteps):
			np.random.shuffle(ids)

			batch = 0
			sum = 0					
			for i in range(N): 
				id = ids[i]
				p = self.Predict(augmentedData[id])
				sum += costFunc.Compute(p,labels[id]) #have to call cost func first to populate node.Y (forward-pass)
				dCdP = costFunc.Gradient(p,labels[id]) #computes gradient with respect to the prediction
				self.Layers[-1].FinalLayerGradient(dCdP,self.Layers[-2].OutputVector)

				for j in np.arange(len(self.Layers)-2,-1,-1):
					if j > 0:
						self.Layers[j].MiddleLayerGradient(self.Layers[j+1],self.Layers[j-1].OutputVector)
					else:
						self.Layers[j].MiddleLayerGradient(self.Layers[j+1],augmentedData[id])
				batch+=1
				if batch == int(batchSize+1):
					batch = 0
					for layer in self.Layers:
						layer.UpdateWeights(alpha)
			if batch!= 0:
				for layer in self.Layers:
					layer.UpdateWeights(alpha)
			meanC += sum
			q+=1
			batchSize += 1
			if steps % 10 == 0:
				
				meanC /= q
				q = 0
				if prevC is not None:
					if meanC < prevC:
						print("Improved at",steps,"C=",meanC,batchSize,alpha)
						alpha = min(0.2,alpha*1.1)
					else:
						print("Deproved at",steps,"C=",meanC,batchSize,alpha)
						alpha = max(1e-5,alpha*0.7)
						
				prevC = meanC*1.0
			
		return

## Testing the Network

In [ ]:
names, labels, data = LoadCategoricalData("../Data/cuteness.dat")
network = Network(2,1)
network.AddLayer(Layer(8,ReluNode))
network.AddLayer(Layer(1,SigmoidNode))

network.Train(data,labels,CostFunction(),1000)

ScanClassifierSpace(network,0,10,0,6,100)
pt.clim([-0.2,1.2])
PlotData(data,labels,names)

In [ ]:
names, labels, data = LoadCategoricalData("../Data/cuteness_augmented.dat")
network = Network(2,1)
network.AddLayer(Layer(4,SigmoidNode))
network.AddLayer(Layer(13,ReluNode))
network.AddLayer(Layer(1,SigmoidNode))

network.Train(data,labels,ProbCost(),2000)

ScanClassifierSpace(network,0,10,0,6,100)
pt.clim([-0.2,1.2])
PlotData(data,labels,names)

In [ ]:
names, labels, data = LoadCategoricalData("../Data/complex_classifier.dat")
network = Network(2,1)

network.AddLayer(Layer(8,ReluNode))
network.AddLayer(Layer(13,SigmoidNode))
network.AddLayer(Layer(4,ReluNode))
network.AddLayer(Layer(1,SigmoidNode))

network.BatchTrain(data,labels,ProbCost(),1000)

ScanClassifierSpace(network,-6,6,-6,6,300,3)
pt.clim([-0.2,1.2])
# PlotData(data,labels)


In [ ]:
##Need to use this on complex_function as the values are non-integers.
def LoadNumericalData(file):
	names = []
	labels = []
	data = []
	with open(file) as file:
		for line in file:
			entries = line.rstrip().split(' ')
			names.append(entries[0])
			labels.append(float(entries[1]))
			data.append(np.array([ float(entries[i]) for i in range(2,len(entries))]))
	return names, labels, data



names, labels, data = LoadNumericalData("../Data/complex_function.dat")
network = Network(2,1)
network.AddLayer(Layer(8,ReluNode))
network.AddLayer(Layer(16,ReluNode))
network.AddLayer(Layer(8,ReluNode))
network.AddLayer(Layer(13,ReluNode))
network.AddLayer(Layer(4,ReluNode))
network.AddLayer(Layer(1))
network.BatchTrain(data,labels,CostFunction(),1240)


ScanClassifierSpace(network,-6,6,-6,6,100,4,False)
pt.colorbar()
pt.plot()



## Modified Cost Functions

The following cost function uses logarithms, rather than the least squares. It should make the optimiser converge faster by prioritising 'very wrong' answers over 'slightly wrong'.

In [ ]:
class ProbCost:
	def Compute(self,prediction,label):
		if label == 1:
			return -np.log(prediction+0.00000001) ## give an offset so you never accidentally compute log(0) 
		else:
			return -np.log(1.00000001 - prediction)
	
	def Gradient(self,prediction,label):
		if label == 1:
			return -1.0/(prediction+0.00000001)
		else:
			return 1.0/(1.00000001 - prediction)
		
names, labels, data = LoadCategoricalData("../Data/cuteness_augmented.dat")
network = Network(2,1)
network.AddLayer(Layer(8,ReluNode))
network.AddLayer(Layer(16,SigmoidNode))
network.AddLayer(Layer(1,SigmoidNode))

network.Train(data,labels,ProbCost(),1000)

ScanClassifierSpace(network,0,10,0,6,100)
pt.clim([-0.2,1.2])
PlotData(data,labels,names)


names, labels, data = LoadCategoricalData("../Data/complex_classifier.dat")
network = Network(2,1)
network.AddLayer(Layer(8,ReluNode))
network.AddLayer(Layer(1,SigmoidNode))

network.Train(data,labels,CostFunction(),1000)

ScanClassifierSpace(network,-6,6,-6,6,100)
pt.clim([-0.2,1.2])